# Assembling the StrokeMRI dataset

In [ ]:
import os       # using operating system dependent functionality (folders)
import pandas as pd # data analysis and manipulation
import numpy as np    # numerical computing (manipulating and performing operations on arrays of data)
import matplotlib.pyplot as plt
import glob

import sys
sys.path.insert(0, '../') # path to functions
import cvasl.seperated as sep
from cvasl.file_handler import Config


## Read data into pandas dataframe

How do we define which files should be stitched together?
options:
- all files in folder
- based on suffix (e.g. "n=895_06-Feb-2023_PVC2.tsv")
- check first columns to see whether it matches

For now, we will will use the first option

In [ ]:
experiment_folder= 'StrokeMRI_correct'

In [ ]:
config = Config()
root_directory = config.get_directory('raw_data')
if os.path.isdir(os.path.join(root_directory, experiment_folder)):
    print("this folder exists, we will take tsv from here")
else: 
    print("this folder does not seem to exist, try typing again")

In [ ]:
root_directory = config.get_directory('raw_data')
tsv_path = os.path.join(root_directory, experiment_folder)

tsv_files = [os.path.join(tsv_path, file) for file in os.listdir(tsv_path) if file.endswith('.tsv')]
tsv_files

In [ ]:
# Read files into dataframes
dataframes = [pd.read_csv(file, sep='\t', header=[0,1], index_col=0) for file in tsv_files]
sample_df = dataframes[0]
cols = sample_df.columns
sample_df

## Copy identical columns from any file

we could also read it from the data, but if it's always the same, we can just define it here

In [ ]:
# new method
## will be recoded with function call from main branch

def check_identical_columns(tsv_path):
    """
    Here we enter the path to a folder, then return which columns which in
    all files are exactly duplicated.In name and values
    """
    tsv_files = glob.glob(os.path.join(tsv_path, '*.tsv'))
    dataframes = [
        pd.read_csv(file, sep='\t', header=[0, 1], index_col=0)
        for file in tsv_files
    ]
    key_df, *rest_dfs = dataframes

    shared_columns = set(key_df.columns)

    for frame in rest_dfs:
        # check which labels are shared
        shared_columns = shared_columns.intersection(frame.columns)

    result = []
    for column in shared_columns:
        for frame in rest_dfs:
            if not frame[column].equals(key_df[column]):
                break
        else:
            result.append(column)
    return result

identical_columns = check_identical_columns(tsv_path)

In [ ]:
identical_columns

In [ ]:
stitched = sample_df[identical_columns].copy()

In [ ]:
n_identical = stitched.shape[1]

In [ ]:
n_identical

In [ ]:


stitched = sample_df[cols[:n_identical]].copy()
stitched['renumber'] = stitched.index


## Add unique columns from files

In [ ]:
for df in dataframes:
    for col in df.columns[n_identical:]:
        stitched[col] = df[col]

stitched.columns

## Add sex and age data

In [ ]:
sexage_path = os.path.join(root_directory, 'age_data', 'Age_Sex_StrokeMRI.csv')
sexage_df = pd.read_csv(sexage_path, index_col=0)
sexage_df['renumber'] = sexage_df.index
sexage_df['renumber'] = sexage_df['renumber'].apply(str)

#sexage_df['renumber']
# tp 2 then + '2_1' if 1 then 1_1

In [ ]:
sexage_df

In [ ]:
stitched

In [ ]:
# needs to be recoded

In [ ]:
sexage_df.loc[sexage_df.TP == 1, 'add_column'] = "_1"
sexage_df.loc[sexage_df.TP == 2, 'add_column'] = "_2"
sexage_df['renumber'] = sexage_df['renumber'] + sexage_df['add_column']

sexage_df['renumber'] = 'sub-'+ sexage_df['renumber'] 

Now we need to reformat the participant ID

In [ ]:
sexage_df

In [ ]:
stitched = stitched.reset_index(drop=True)
sexage_df = sexage_df.reset_index(drop=True)
result = pd.concat([stitched, sexage_df], axis=1, join="inner")
result

In [ ]:
# for col in sexage_df:
#     stitched[col] = sexage_df[col]

result.columns = [c[0]  for c in result.columns]
result.columns

In [ ]:
result

In [ ]:
result = result.rename(columns={"A": "Age", "S": "Sex"})


## save off file

In [ ]:
 
filepath = '../open_work/internal_results/StrokeMRI_stitched.csv' 
result.to_csv(filepath)  

# Look at columns

In [ ]:
result.columns

In [ ]:
result.session.unique()

### SO here there was no need to filter on session. We got lucky